#Automação de Relatórios de Vendas e Envio de E-mails com Python

##1: Importação das bibliotecas e base de dados

In [ ]:
import pandas as pd
import smtplib
import email.message

tabela_vendas = pd.read_excel("Vendas.xlsx")
tabela_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


Nesta etapa é realizada a importação da base de vendas que será utilizada para gerar os relatórios automatizados.

##2: Cálculo do faturamento por loja

In [ ]:
tabela_faturamento = (
    tabela_vendas[['ID Loja', 'Valor Final']]
    .groupby('ID Loja')
    .sum()
    .sort_values(by='Valor Final', ascending=False)
)

display(tabela_faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


Aqui calculamos o faturamento total de cada loja, permitindo identificar quais geram maior receita.

##3: Cálculo da quantidade de produtos vendidos por loja

In [ ]:
tabela_quantidade = (
    tabela_vendas[['ID Loja','Quantidade']]
    .groupby('ID Loja')
    .sum()
    .sort_values(by='Quantidade', ascending=False)
)

display(tabela_quantidade)

,Quantidade
ID Loja,
Shopping Vila Velha,9224
Shopping União de Osasco,8730
Shopping SP Market,8927
Shopping Recife,8581
Shopping Morumbi,8508
Shopping Midway Mall,8206
Shopping Iguatemi Fortaleza,8629
Shopping Ibirapuera,8723
Shopping Eldorado,8719


Essa métrica indica o volume de vendas de cada unidade.

##4: Cálculo do ticket médio por loja

In [ ]:
ticket_medio = (tabela_faturamento['Valor Final'] / tabela_quantidade['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Médio'})
ticket_medio = ticket_medio.sort_values(by='Ticket Médio', ascending=False)

display(ticket_medio)

,Ticket Médio
ID Loja,
Iguatemi Esplanada,198.098019
Iguatemi Campinas,197.248909
Bourbon Shopping SP,194.754598
Parque Dom Pedro Shopping,194.519552
Rio Mar Recife,194.377299
Shopping Iguatemi Fortaleza,194.092479
Shopping Midway Mall,193.814404
Center Shopping Uberlândia,193.453228
Ribeirão Shopping,193.441586


O ticket médio mostra quanto, em média, cada cliente gasta por compra em cada loja.

##5: Consolidação do relatório completo

In [ ]:
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
display(tabela_completa)

Aqui consolidamos todas as métricas em uma única tabela que será utilizada no envio automático de relatórios.

##6: Função de envio automático de e-mail

In [ ]:
import os

def enviar_email(resumo_loja, loja):
    server = smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()

    email_origem = os.getenv("EMAIL_USER")
    senha = os.getenv("EMAIL_PASSWORD")

    corpo_email = f"""
    <p>Prezados,</p>
    <p>Segue relatório de vendas da loja <strong>{loja}</strong>:</p>
    {resumo_loja.to_html()}
    <p>Qualquer dúvida estou à disposição.</p>
    """

    msg = email.message.Message()
    msg['Subject'] = f"Relatório de vendas - {loja}"
    msg['From'] = email_origem
    msg['To'] = email_origem
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email)

    server.login(email_origem, senha)
    server.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    server.quit()

    print(f"E-mail enviado para {loja}")

A função encapsula toda a lógica de envio automático de relatórios por e-mail utilizando SMTP.

##7: Envio do relatório geral para a diretoria

In [ ]:
enviar_email(tabela_completa, "Todas as lojas")

Email enviado


##8: Envio automático para cada loja

In [ ]:
lista_lojas = tabela_vendas['ID Loja'].unique()

for loja in lista_lojas:
    tabela_loja = tabela_vendas.loc[tabela_vendas['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
    resumo_loja = tabela_loja.groupby('ID Loja').sum()
    resumo_loja['Ticket Médio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']
    enviar_email(resumo_loja, loja)


Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado


Esse laço permite o envio automático de relatórios personalizados para cada loja, eliminando trabalho manual.

##Tabela de Insights
Principais Resultados do Projeto:

• Automação completa do processo de geração e envio de relatórios  
• Redução de trabalho manual e risco de erro humano  
• Criação de métricas fundamentais: faturamento, volume de vendas e ticket médio  
• Relatórios personalizados para cada unidade  
• Estrutura facilmente escalável para novos indicadores  